In [1]:
# Notes and stuff:

# Can we predict games?
#  -Compare just teams in general, not players. Then use just champions. Then use players. Then combine all of them.
#  -Use regular season data. Maybe compare each laner individually when looking at players?

# What makes a player good? -need to come up with a heuristic, like wins*games. Test to find the best one.
#  -What is/are the most important values in a player? Is it kda or something less known like cs/m
#  -Look at similarities between people with a higher rating based on our heuristic

# Role importance?
#  -Does a single role have more influence over a team's win. For example do teams with a better mid laner win more often.
#  -Need the rating system first

# DATA:

# Player data from Worlds -> playersWorlds

# Champion data from Worlds play-ins -> champs

# Player data from regular season by region -> reg
#   reg[0] --> North America (NA)
#   reg[1] --> Europe (EU)
#   reg[2] --> China (LPL)
#   reg[3] --> Korea (LCK)
#   reg[4] --> Taiwan (LMS)
#   reg[5] --> Vietnam (VCS)

# Match data from regular season by region -> matchReg
#   matchReg[0] --> North America (NA)
#   matchReg[1] --> Europe (EU)
#   matchReg[2] --> China (LPL)
#   matchReg[3] --> Korea (LCK)
#   matchReg[4] --> Taiwan (LMS)
#   matchReg[5] --> Vietnam (VCS)

In [1]:
!pip install lxml
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re

# GET requests for all of the different datasets we will need.

# Gather tournament data on all 90 players
r = requests.get("https://lol.gamepedia.com/2018_Season_World_Championship/Main_Event/Player_Statistics")

# Regular season data from North America
r2 = requests.get("https://lol.gamepedia.com/NA_LCS/2018_Season/Summer_Season/Player_Statistics")

# Regular Season data from EU
r3 = requests.get("https://lol.gamepedia.com/EU_LCS/2018_Season/Summer_Season/Player_Statistics")

# Regular Season data from the LPL
r4 = requests.get("https://lol.gamepedia.com/LPL/2018_Season/Summer_Season/Player_Statistics")

# Regular Season data from the LCK
r5 = requests.get("https://lol.gamepedia.com/LCK/2018_Season/Summer_Season/Player_Statistics")

# Regular Season data from the LMS
r6 = requests.get("https://lol.gamepedia.com/LMS/2018_Season/Summer_Season/Player_Statistics")

# Regular Season data from the VCS
r7 = requests.get("https://lol.gamepedia.com/VCS/2018_Season/Summer_Season/Player_Statistics")

# Champion data from play-ins
r8 = requests.get("https://lol.gamepedia.com/2018_Season_World_Championship/Main_Event/Champion_Statistics")

    100% |████████████████████████████████| 5.8MB 117kB/s eta 0:00:01    32% |██████████▌                     | 1.9MB 5.3MB/s eta 0:00:01    67% |█████████████████████▋          | 4.0MB 5.0MB/s eta 0:00:01    91% |█████████████████████████████▎  | 5.3MB 5.3MB/s eta 0:00:01
You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# Match data from each region

# The website changed the way they encoded their data, so the parser for the match data doesn't work anymore, but luckily we
# saved all of the data to a csv file which we will be using instead. But this is how we got the data in the first place from
# their website.

# Match data from North America
r9 = requests.get("https://lol.gamepedia.com/NA_LCS/2018_Season/Summer_Season")

# Match data from Europe
r10 = requests.get("https://lol.gamepedia.com/EU_LCS/2018_Season/Summer_Season")

# Regular Season data from the LPL
r11 = requests.get("https://lol.gamepedia.com/LPL/2018_Season/Summer_Season")

# Regular Season data from the LCK
r12 = requests.get("https://lol.gamepedia.com/LCK/2018_Season/Summer_Season")

# Regular Season data from the LMS
r13 = requests.get("https://lol.gamepedia.com/LMS/2018_Season/Summer_Season")

# Regular Season data from the VCS
r14 = requests.get("https://lol.gamepedia.com/VCS/2018_Season/Summer_Season")

ls = []
matchReg = []
j = 0

# Parse with lxml
ls.append(BeautifulSoup(r9.text, "lxml"))
ls.append(BeautifulSoup(r10.text, "lxml"))
ls.append(BeautifulSoup(r11.text, "lxml"))
ls.append(BeautifulSoup(r12.text, "lxml"))
ls.append(BeautifulSoup(r13.text, "lxml"))
ls.append(BeautifulSoup(r14.text, "lxml"))

# Number of teams for each region
numTeams = [10,10,7,10,8,8]

for bs in ls:
    
    # Different parsing rules for the LPL
    if j==2:
        bs = BeautifulSoup(r11.text, "lxml")

        l = str(bs).split("title=\"Points\"")[1:]

        arr = []  # final array for the dataframe

        # Loop for east and west region
        for t in range(0,2):
            l[t] = l[t].split("title=\"")[1:]

            l2 = []     # Remove unnecessary rows
            l2 = [s for s in l[t] if "std.png" not in s]
            l[t] = l2

            l[t][numTeams[2]-1] = l[t][numTeams[2]-1].split("</div>")[0]

            # Get each team's data
            for x in range(0,numTeams[2]):
                l[t][x] = l[t][x].split("align=\"center")
                l2 = []
                l2.append(l[t][x][0].split("\">")[0])

                # Different parsing rules for each region
                if (t==2):
                    l2.append(l[t][x][5][2:9])
                else:
                    l2.append(l[t][x][3][2:9])

                # add to our final array
                arr.append(l2)

        # Create the dataframe
        matchReg.append(pd.DataFrame(np.array(arr).reshape(len(arr),2), columns = ['Team', 'Record']))
    
    # Need to parse differently for the EU_LCS page
    elif j==1:
        
        l = str(bs).split("Overall")[1].split("</tbody>")[0].split("</th></tr>")[1:-1]
        
        for h in range(0,len(l)):
            l[h] = l[h].split("style=\"background-color:")
            
            # Team name  
            l[h][0] = l[h][0].split("title=")[1].split("\"><img alt")[0][1:]
            
            # Get match data
            for g in range(1,numTeams[j]+1):
                tmpStr = l[h][g][6:]
                tmpStr = 'blank' if tmpStr[0] == 'A' else tmpStr[0:5]
                
                l[h][g] = tmpStr
            
            l[h] = l[h][0:-1]
            
        teams = ["Teams"]
        for t in l:
            teams.append(t[0])
            
        # Create the dataframe
        matchReg.append(pd.DataFrame(np.array(l).reshape(len(l),len(l[0])), columns = teams))

        # Replace our spacer values with NaN to standardize it
        matchReg[j] = matchReg[j].replace("blank",np.nan)
        
    #lms and vcs 
    elif j==4 or j==5:
        l = str(bs).split("Overall")[1].split("</tbody>")[0].split("</th></tr>")[1:-1]
        
        for h in range(0,len(l)):
            l[h] = l[h].split("style=\"background-color:")
            
            # Team name  
            l[h][0] = l[h][0].split("title=")[1].split("\"><img alt")[0][1:]
            
            l2 = []
            l2 = [s for s in l[h] if "display" not in s]
            l[h] = l2
            
            # Get match data
            for g in range(1,numTeams[j]+1):
                tmpStr = l[h][g][6:]
                tmpStr = 'blank' if tmpStr[0] == 'A' else tmpStr[1:6]
                
                l[h][g] = tmpStr
            
            l[h] = l[h][0:-1]
        
            teams = ["Teams"]
        for t in l:
            teams.append(t[0])
            
        # Create the dataframe
        matchReg.append(pd.DataFrame(np.array(l).reshape(len(l),len(l[0])), columns = teams))

        # Replace our spacer values with NaN to standardize it
        matchReg[j] = matchReg[j].replace("blank",np.nan)
      
    # Default parser -> Works for NA_LCS and LCK
    else:
        l = str(bs).split("plainlinks crossbox")[1].split("title=")[1:]

        # Get all of the teams
        for x in range(0,numTeams[j]):
            l[x] = l[x][1:]
            l[x] = l[x].split("\"><img alt")[0]

        # Get all team names as the first element
        l[0:numTeams[j]] = [','.join(l[0:numTeams[j]])]
        l[0] = l[0].split(",")

        # Get match data for each team
        for t in range(1,numTeams[0]+1):
            l[t] = l[t].split("<span class=\"crossbox-match-link")[:-1]

            # Get the score for how well each team did
            for x in range(0,len(l[t])):
                l[t][x] = l[t][x][-5:]

            # Insert the team name at the beginning of the row
            l[t].insert(0,l[0][t-1])

        # Cut off unnecessary rows in the array
        l = l[:numTeams[j]+1]

        # Insert spacing because you can't play yourself
        for t in range(1,numTeams[j]+1):
            l[t].insert(t,'blank')
            l[t] = l[t][:numTeams[j]+1]

        # Insert new column
        l[0].insert(0,'Teams')

        # Create the dataframe
        matchReg.append(pd.DataFrame(np.array(l[1:]).reshape(len(l)-1,len(l[0])), columns = l[0]))

        # Replace our spacer values with NaN to standardize it
        matchReg[j] = matchReg[j].replace("blank",np.nan)
    
    j = j+1


ValueError: cannot reshape array of size 20 into shape (10,11)

In [127]:
# Since we stored the data into csv files, we can just read them in with pandas.
matchReg = []
matchReg.append(pd.read_csv("naMatchdat.csv").drop('Unnamed: 0',axis=1))
matchReg.append(pd.read_csv("EUMatchdat.csv").drop('Unnamed: 0',axis=1))
matchReg.append(pd.read_csv("LPLMatchdat.csv").drop('Unnamed: 0',axis=1))
matchReg.append(pd.read_csv("LCKMatchdat.csv").drop('Unnamed: 0',axis=1))
matchReg.append(pd.read_csv("LMSMatchdat.csv").drop('Unnamed: 0',axis=1))
matchReg.append(pd.read_csv("VCSMatchdat.csv").drop('Unnamed: 0',axis=1))

# Some data manipulation to make our lives easier down the road
for x in range(0,len(matchReg)):
    matchReg[x]['Winner'] = matchReg[x]['Winner']*100
    t = matchReg[x].replace(0,5)
    t = t.replace(100,0)
    matchReg[x] = t.replace(5,100)
    
# Combine each region's data into one master dataframe
masterMatch = pd.concat(matchReg).reset_index().drop('index',axis=1)

In [6]:
# Player data for the Worlds Tournament

# Parse with lxml
bs = BeautifulSoup(r.text, "lxml")

# split to get all 90 players individually
players = str(bs).split("std.png/45px")[1:]

# split to get each column
for x in range (0,len(players)):
    players[x] = players[x].split("</td>")
    
    # Get the team name
    players[x][0] = players[x][0][1:].split("_std.png")[0][:-4]
    
    # Get the player's name
    players[x][1] = players[x][1].split("title=\"")[1].split("\">")[0]
    
    for y in range (2,19):
        if y!=17:
            players[x][y] = players[x][y].split("center\">")[1].split("\n")[0]
    
    players[x][17] = players[x][17][-17]

# Convert into data frame with appropriate columns
playersWorlds = pd.DataFrame(np.array(players).reshape(90,20), columns = ['Team','Player','Games','Wins','Losses','Winrate','Kills','Deaths','Assists','KDA','CS','CSPM','Gold','GPM','Kill Participation','Kill Share','Gold Share','Champions Played','19','20'])

# Drop unnecessary columns
playersWorlds = playersWorlds.drop(['19','20'],axis=1)

# Get numeric data
playersWorlds['Games'] = playersWorlds['Games'].str.extract('(\d+)', expand=False)
playersWorlds['Gold'] = playersWorlds['Gold'].str[:-1]
playersWorlds['Kill Participation'] = playersWorlds['Kill Participation'].str[:-1]
playersWorlds['Kill Share'] = playersWorlds['Kill Share'].str[:-1]
playersWorlds['Gold Share'] = playersWorlds['Gold Share'].str[:-1]

#playersWorlds

In [4]:
# Regular season data for each region
# We also took precautions and decided to save this to a csv file once we combined all of the data together from each region.

ls = []
reg = []

# Parse with lxml
ls.append(BeautifulSoup(r2.text, "lxml"))
ls.append(BeautifulSoup(r3.text, "lxml"))
ls.append(BeautifulSoup(r4.text, "lxml"))
ls.append(BeautifulSoup(r5.text, "lxml"))
ls.append(BeautifulSoup(r6.text, "lxml"))
ls.append(BeautifulSoup(r7.text, "lxml"))

j = 0
for bs in ls:
    
    # split to get all the players individually
    players = str(bs).split("std.png/45px")[1:]

    # split to get each column
    for x in range (0,len(players)):
        players[x] = players[x].split("</td>")

        # Get the player's name
        players[x][1] = players[x][1].split("title=\"")[1].split("\"")[0]

        # Get the team name
        players[x][0] = players[x][0][1:].split("_std.png")[0][:-4]

        for y in range (2,17):
            players[x][y] = players[x][y].split("center\">")[1].split('\n')[0]
            
        players[x][17] = players[x][17][-18:-16]

    # Convert into data frame
    reg.append(pd.DataFrame(np.array(players).reshape(len(players),len(players[0])), columns = ['Team','Player','Games','Wins','Losses','Winrate','Kills','Deaths','Assists','KDA','CS','CSPM','Gold','GPM','Kill Participation','Kill Share','Gold Share','Champions Played','19','20']))

    # Drop unnecessary columns
    reg[j] = reg[j].drop(['19','20'],axis=1)
    
    # Get just numeric data
    reg[j]['Games'] = reg[j]['Games'].str.extract('(\d+)', expand=False)
    reg[j]['Champions Played'] = reg[j]['Champions Played'].str.extract('(\d+)', expand=False)
    reg[j]['Gold'] = reg[j]['Gold'].str[:-1]
    reg[j]['Gold Share'] = reg[j]['Gold Share'].str[:-1]
    reg[j]['Kill Participation'] = reg[j]['Kill Participation'].str[:-1]
    reg[j]['Kill Share'] = reg[j]['Kill Share'].str[:-1]
        
    j = j+1

# reg contains regular season data by region
# reg[0] --> North America (NA)
# reg[1] --> Europe (EU)
# reg[2] --> China (LPL)
# reg[3] --> Korea (LCK)
# reg[4] --> Taiwan (LMS)
# reg[5] --> Vietnam (VCS)

# Convert data into numeric form
for t in reg:
    k=0
    for col in t.columns:
        if k>1 and k<18:
            t[col] = pd.to_numeric(t[col])
        k=k+1
        
# Sort the tables based on 'Games' and 'Player' to standardize the view
reg[0] = reg[0].sort_values(['Games','Player'],ascending=[False,True])
reg[1] = reg[1].sort_values(['Games','Player'],ascending=[False,True])
reg[2] = reg[2].sort_values(['Games','Player'],ascending=[False,True])
reg[3] = reg[3].sort_values(['Games','Player'],ascending=[False,True])
reg[4] = reg[4].sort_values(['Games','Player'],ascending=[False,True])
reg[5] = reg[5].sort_values(['Games','Player'],ascending=[False,True])

# Get Position data for players from the regular season.
# Position can be Top, JG, Mid, ADC, or Supp.

# NOTE: This was hard-coded because the values won't change and it was more feasible than downloading 30 separate tables 
#       and just looking at a single column for each one.

#pos = ['Top','ADC','Mid','JG','Top','Supp','Mid','Supp','ADC','Top','ADC','JG','JG','Mid','Top','Supp','Mid','Top','JG','JG','Mid','Mid','Supp','ADC','ADC','Top','Supp','ADC','Top','Top','Supp','ADC','Mid','Top','Supp','ADC','JG','Supp','Supp','JG','Mid','JG','Mid','JG','Top','Mid','ADC','Supp','ADC','ADC','JG','JG','Supp','Mid','JG','Mid','ADC','Top','Supp','JG','Supp','JG','ADC','Supp','Mid','JG','Top']
pos = ['Supp', 'Mid', 'ADC', 'JG', 'Top', 'JG', 'Top', 'Top', 'Mid',
       'Supp', 'Mid', 'JG', 'Top', 'ADC', 'ADC', 'ADC', 'Supp', 'Supp',
       'JG', 'ADC', 'ADC', 'Mid', 'Mid', 'Top', 'Top', 'Top', 'Supp',
       'Supp', 'Mid', 'Mid', 'Top', 'ADC', 'JG', 'Top', 'JG', 'ADC',
       'Supp', 'Supp', 'JG', 'Mid', 'JG', 'Supp', 'JG', 'Mid', 'Top',
       'Mid', 'Supp', 'ADC', 'ADC', 'ADC', 'JG', 'JG', 'Supp', 'Mid',
       'Mid', 'JG', 'Top', 'ADC', 'Supp', 'Supp', 'Mid', 'JG', 'JG', 'ADC',
       'Supp', 'JG', 'Top']
reg[0]['Position'] = pos

#pos1 = ['Mid','Mid','Supp','Supp','Top','Supp','JG','ADC','Top','Top','JG','ADC','Mid','ADC','Mid','JG','Supp','ADC','JG','Mid','Mid','Mid','ADC','Mid','ADC','JG','Mid','Top','Supp','Supp','Top','Supp','Supp','Top','Supp','Top','ADC','Top','JG','JG','Top','JG','ADC','Supp','ADC','Top','Mid','JG','JG','ADC','JG','JG','ADC','ADC','Mid','Supp']
#pos1 = ['JG','ADC','Supp','Mid','Supp','Top','Supp','JG','ADC','Top','Top','Mid','Mid','ADC','JG','Mid','Mid','Mid','ADC','JG','Mid','Mid','ADC','Mid','ADC','JG','Top','Supp','Top','JG','Supp','JG','Top','JG','Top','Top','Supp','Supp','Supp','ADC','Top','Supp','ADC','ADC','Supp','Top','Mid','JG','ADC','JG','JG','ADC','JG','ADC','Mid','Supp']
pos1 = ['JG', 'ADC', 'Top', 'ADC', 'Supp', 'JG', 'Mid', 'Mid', 'Mid', 'ADC',
       'Supp', 'Top', 'Supp', 'Top', 'Top', 'Mid', 'Mid', 'JG', 'Mid',
       'JG', 'Mid', 'ADC', 'ADC', 'Supp', 'Supp', 'ADC', 'JG', 'JG', 'JG',
       'Mid', 'Supp', 'Top', 'Top', 'Supp', 'Top', 'Mid', 'Supp', 'Top',
       'ADC', 'Supp', 'Top', 'JG', 'ADC', 'ADC', 'Supp', 'Top', 'Mid',
       'JG', 'JG', 'JG', 'ADC', 'JG', 'ADC', 'ADC', 'Supp', 'Mid']
reg[1]['Position'] = pos1

#pos2 = ['Supp','ADC','JG','Supp','Top','Mid','Top','Mid','Supp','JG','JG','ADC','Mid','Top','Top','Mid','Mid','JG','ADC','Supp','Supp','ADC','ADC','Top','Supp','ADC','Supp','Supp','Top','Top','ADC','Mid','Mid','Mid','Top','Supp','ADC','Mid','JG','JG','Top','Top','JG','Mid','Supp','Supp','Supp','Top','ADC','Supp','JG','Mid','Top','JG','ADC','Mid','ADC','Mid','ADC','Supp','Mid','JG','JG','ADC','ADC','JG','Top','JG','ADC','JG','Mid','Top','JG','JG','Top','ADC','Mid','JG','JG','Supp','JG','Top','JG','ADC','ADC','Top','Mid','JG','JG','Mid','ADC','ADC','Top','Mid','Top','ADC','Supp','Supp','ADC','JG','Supp','Mid']
pos2 = ['JG','ADC','Supp','Mid','Top','JG','Mid','Top','Top','Supp','ADC','Supp','Mid','Supp','Top','ADC','JG','JG','Mid','Supp','Mid','Supp','Top','Mid','ADC','Supp','ADC','Supp','Top','Mid','ADC','Top','ADC','JG','Supp','Top','Supp','Mid','ADC','Supp','Top','Mid','Top','JG','Supp','ADC','JG','Mid','Top','JG','ADC','JG','JG','Supp','Top','ADC','Mid','JG','Top','Mid','JG','ADC','ADC','Mid','ADC','Mid','JG','Supp','Top','JG','Top','Mid','JG','Top','Mid','JG','Top','ADC','ADC','JG','ADC','JG','Mid','JG','Supp','JG','ADC','Mid','Supp','ADC','Top','JG','JG','Top','ADC','Mid','ADC','ADC','JG','Supp','Mid','Supp']
reg[2]['Position'] = pos2

#pos3 = ['Supp','Mid','Top','ADC','Supp','Top','Supp','Mid','Supp','Top','ADC','Top','ADC','Mid','Supp','Supp','Top','Mid','JG','Mid','ADC','Supp','Top','JG','Supp','Mid','JG','Mid','Top','Top','Top','Mid','ADC','Top','ADC','Supp','ADC','JG','ADC','ADC','JG','ADC','JG','Mid','Supp','JG','JG','Mid','JG','JG','JG','JG','ADC','JG','JG','Mid','Supp','JG','ADC','JG','Supp','ADC','JG','Mid','ADC','Mid','Top','Top','ADC','Supp','Top']
pos3 = ['Top', 'Mid', 'Supp', 'Supp', 'Mid', 'Top', 'Supp', 'ADC', 'JG',
       'Top', 'ADC', 'Supp', 'Mid', 'ADC', 'Top', 'Mid', 'Top', 'Top',
       'ADC', 'Supp', 'Supp', 'Supp', 'ADC', 'Mid', 'Top', 'Mid', 'ADC',
       'Mid', 'Mid', 'Supp', 'Top', 'JG', 'JG', 'ADC', 'JG', 'Top', 'JG',
       'ADC', 'Supp', 'ADC', 'Top', 'Supp', 'JG', 'JG', 'JG', 'ADC', 'Mid',
       'JG', 'JG', 'Mid', 'Mid', 'JG', 'JG', 'ADC', 'JG', 'Supp', 'Mid',
       'JG', 'JG', 'Supp', 'JG', 'ADC', 'ADC', 'JG', 'Mid', 'ADC', 'Supp',
       'Top', 'ADC', 'Top', 'Top']
reg[3]['Position'] = pos3

#pos4 = ['Supp','Top','Mid','JG','Supp','Top','Top','JG','Supp','Supp','ADC','Supp','Mid','ADC','ADC','Mid','JG','Mid','Top','Supp','ADC','ADC','Mid','ADC','JG','Mid','ADC','Top','Supp','Supp','ADC','Top','JG','Mid','Top','JG','Mid','JG','ADC','Top','JG','Top','JG','ADC','JG','Mid','Top','JG','JG','Top','JG','Top','Supp','ADC','JG','Supp','Supp','Supp','JG','JG']
pos4 = ['Supp', 'Top', 'JG', 'Mid', 'Top', 'Supp', 'ADC', 'Mid', 'ADC',
       'Top', 'JG', 'Supp', 'Supp', 'JG', 'Mid', 'ADC', 'Supp', 'ADC',
       'Supp', 'ADC', 'Mid', 'Top', 'ADC', 'Mid', 'JG', 'Mid', 'Top',
       'ADC', 'Supp', 'Supp', 'Top', 'ADC', 'Top', 'JG', 'Mid', 'Mid',
       'JG', 'JG', 'ADC', 'Top', 'JG', 'Top', 'ADC', 'JG', 'JG', 'Mid',
       'Top', 'JG', 'JG', 'JG', 'Top', 'ADC', 'Top', 'Supp', 'JG', 'Supp',
       'Supp', 'Supp', 'JG', 'JG']
reg[4]['Position'] = pos4

#pos5 = ['Top','ADC','Supp','Mid','ADC','Mid','Top','Top','Mid','Supp','Mid','JG','JG','JG','ADC','Mid','ADC','Top','Mid','Top','Supp','JG','ADC','Top','ADC','Supp','JG','ADC','Supp','JG','JG','Supp','Supp','Mid','Top','ADC','ADC','JG','Supp','Mid','Top','JG','Supp','Mid','Top','Top','Supp','Mid','JG','ADC','Supp','Supp','Supp','JG','Mid','Supp','JG','Top','JG','Mid']
pos5 = ['ADC', 'Mid', 'Supp', 'Top', 'ADC', 'Top', 'JG', 'Mid', 'Supp',
       'ADC', 'JG', 'Top', 'Mid', 'Mid', 'JG', 'Mid', 'ADC', 'Top', 'Supp',
       'Mid', 'Top', 'JG', 'Top', 'ADC', 'Supp', 'ADC', 'JG', 'ADC',
       'Supp', 'JG', 'Supp', 'JG', 'Supp', 'Top', 'Mid', 'ADC', 'ADC',
       'JG', 'Supp', 'Mid', 'Top', 'JG', 'Supp', 'Mid', 'Top', 'Top',
       'Supp', 'Mid', 'JG', 'ADC', 'Supp', 'Supp', 'Supp', 'Mid', 'JG',
       'Supp', 'JG', 'Top', 'JG', 'Mid']
reg[5]['Position'] = pos5

# Combine regular season player data from each region into a master table
masterReg = pd.concat(reg).sort_values(['Games','Player'],ascending=[False,True]).reset_index().drop('index',axis=1)
#masterReg.to_csv('masterReg.csv')  # Save the data for use later on so we don't have to parse every time.

In [17]:
# Read in the table since we saved it already.
masterReg = pd.read_csv("masterReg.csv").drop("Unnamed: 0",axis=1)

# List of all column vars
vars = ['KDA', 'GPM', 'Kill Share', 'Kill Participation', 'Kills', 'Deaths', 'Assists','Gold', 'Gold Share', 'CS', 'CSPM', 'Champions Played']

# Create tables based on player position for our machine learning model.
top = masterReg[masterReg['Position']=='Top']
jg = masterReg[masterReg['Position']=='JG']
mid = masterReg[masterReg['Position']=='Mid']
adc = masterReg[masterReg['Position']=='ADC']
supp = masterReg[masterReg['Position']=='Supp']

In [5]:
# Champion data from worlds

# Parse with lxml
bs = BeautifulSoup(r8.text, "lxml")

# split to get all champions individually
tmp = str(bs).split("Square.png/40px")[1:]

# split to get each column
for x in range (0,len(tmp)):
    
    tmp[x] = tmp[x].split("</td>")
    
    # Get the Champion's name
    tmp[x][0] = tmp[x][0].split("title=\"")[1].split("\"")[0]
    
    # Get other column data
    for y in range (1,19):
        if y==5 or y==6:
            tmp[x][y] = tmp[x][y].split("</b>")[0]
        if y != 7:
            tmp[x][y] = tmp[x][y].split("center\">")[1].split("\n")[0]
    
    t = tmp[x][7].split("_blank\">") 
    tmp[x][7] = 0 if len(t)==1 else t[1][:2]

# Convert into data frame with appropriate columns
champs = pd.DataFrame(np.array(tmp).reshape(len(tmp),len(tmp[0])), columns = ['Champion','Bans','Games','Wins','Losses','Winrate','PB','By','Kills','Deaths','Assists','KDA','CS','CSPM','Gold','GPM','Kill Participation','Kill Share','Gold Share', '20'])

# Drop unnecessary columns
champs = champs.drop('20',axis=1)

# Get just numeric data
champs['By'] = champs['By'].str.extract('(\d+)', expand=False)
champs['Gold'] = champs['Gold'].str[:-1]
champs['Kill Participation'] = champs['Kill Participation'].str[:-1]
champs['Kill Share'] = champs['Kill Share'].str[:-1]
champs['Gold Share'] = champs['Gold Share'].str[:-1]
champs['Winrate'] = champs['Winrate'].str[3:]
champs['PB'] = champs['PB'].str[3:]

# Making data NaN for champs banned, but not played. Might want to drop these entries entirely though.
pat = re.compile('-1')
for col in champs.columns:
    champs[col] = champs[col].replace(pat, np.nan)

# Save the data as a csv so we don't have to parse it every time.
champs.to_csv("champs.csv")

In [ ]:
# The first bit of machine learning we want to do is determine what makes a player good. To do this, we will be using winrate
# as our y value, and all of the different player statistics as our x variables. The results will tell us which of the
# different variables are important and how important each one is individually. 

# First, we separated all of the data into their own positions, because what makes a mid laner good is probably different from
# what makes a support good.

# Next, we have to determine which variables we should be using to figure out their weights, or coefficients. To do this, we
# created a powerset function that returns every possible combination of the variables we can have.

In [7]:
# The code for this function was taken from:
#     https://codereview.stackexchange.com/questions/178225/computing-the-powerset-of-a-list

from itertools import chain
from itertools import combinations

# Powerset function of a given list
def gen_powerset(l):
    if not l:  # List is empty
        yield []
        return
    for sub_powerset in gen_powerset(l[1:]): # Generate next list
        yield sub_powerset
        yield [l[0]] + sub_powerset

In [ ]:
# Next, we removed outliers from the data that would significantly impact the results. This is due to us using winrate as our
# measure of how good a player is. For example, if we have a substitute player that only played one game, and won, our algorithm
# thinks that he is the best player in the world. To remove outliers, we tested each dataset with at least 'x' games played
# by players to see what the cutoff should be. We don't want to remove more than 10 games, but that would encrouching on the
# regular season starters for some regions, so we looked at 0-10 for each region.

# The code below is linear regression in machine learning for predicitive analysis.

# Source on what linear regression is and why we use it
#  https://towardsdatascience.com/linear-regression-using-python-ce21aa90ade6
  
# Example of basic linear regression
#  https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html

# Source on linear regression of multi-variables, which is the type we did
#  https://datatofish.com/multiple-linear-regression-python/

# Source to understand the purpose and basic use of test and training data
#  https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

In [9]:
# This is an example of how we determined the cutoffs for each position. This was done on the adc table.

from sklearn import linear_model
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Must have played at least 0-10 games.
for u in range(0,10):
    
    # All of the variables we want to look at
    vars = ['KDA', 'GPM', 'Kill Share', 'Kill Participation', 'Kills', 'Deaths', 'Assists','Gold', 'Gold Share', 'CS', 'CSPM']

    # Must have played at least 'u' games
    tmp = adc[adc['Games'] >= u]

    best = -1    # Keeps track of the best variance
    bestLs = []  # Holds the list of variables for the best variance
    coef = []    # Contains the coefficients for the variables
    
    # Set winrate to be our ranking for the players
    y = tmp['Winrate']

    # Look at every possible combination of variables for the players
    for ps in gen_powerset(vars):
        
        if ps: # Dont include empty list
            
            X = tmp[ps]
            
            # Split data into test and training sets with an 80/20 split, which is standard to minimize overfitting.
            # 80% of the data is used for training and the remaining 20% is used for testing.
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
            
            # Fit the model to the training data
            regr = linear_model.LinearRegression()
            regr.fit(X_train, y_train)

            #numpy array that contains all the predicted values for the input values in the X series.
            y_pred = regr.predict(X_test)

            # Get the variance score between the test data and our prediction. 1 would be a perfect score.
            residuals = r2_score(y_test, y_pred)

            # Keep track of the best score and which variables we used.
            if residuals > best:
                best = residuals
                bestLs = ps
                coef = regr.coef_
                
    print(str(u) + ": " + str(best))

0: 0.893988370271
1: 0.893988370271
2: 0.875577693052
3: 0.847706284473
4: 0.904623303699
5: 0.815746226433
6: 0.933270580443
7: 0.933270580443
8: 0.927521733627
9: 0.911363927907


In [50]:
# From this output, we see that if we look at just players that have played at least 6 games our variance goes all the way
# up to a 93% accuracy. If we used every player, so at least 0 games played, we would have an 89% accuracy. In the end, this
# should help our model be more consistent by removing outliers in the data. We're allowed to do this because it makes sense 
# heuristically, and we still have a sizeable portion of the regular data. This removes the problem of having a person that
# went 1/1 in their games and our algorithm tries to use them as the best player.

# By having 10 be our max we want to cut out, it ensures that we keep starters for each region, as well as players that split
# time like the junglers for Cloud9 Svenskeren and Blaber. The only players we remove are substitutes.

# This shows us that we are still using 86% of the main data, so we aren't cutting out a significant portion.
len(adc[adc['Games']>=6])/len(adc)

0.8674698795180723

In [26]:
# We repeated this process for each position and came up with the following numbers for minimum games played:
#   top -  7
#   jg -   2
#   mid -  4
#   adc -  6
#   supp - 7

top = top[top['Games']>=7]
jg = jg[jg['Games']>=2]
mid = mid[mid['Games']>=4]
adc = adc[adc['Games']>=6]
supp = supp[supp['Games']>=7]

# The next step is to look at each position and figure out which variables we want to use now that we removed our outliers.

In [10]:
# TOP MODEL

from sklearn import linear_model
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Look at the top 20 results
numResults = 20

# Must have played at least 7 games
tmp = top[top['Games'] >= 7]

#These are the possible column variables to be tested in combinations
vars = ['KDA', 'GPM', 'Kill Share', 'Kill Participation', 'Kills', 'Deaths', 'Assists','Gold', 'Gold Share', 'CS', 'CSPM']

best = [-1]*numResults   # Array of top 20 variance scores
bestLs = [-1]*numResults # Array of corresponding variables to each of the variance scores
coef = [[-1]]*numResults # Array of weights associated with each of the variables producing the variance scores

y = tmp['Winrate']

for ps in gen_powerset(vars):
    if ps:
        X = tmp[ps]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        regr = linear_model.LinearRegression()
        regr.fit(X_train, y_train)
        y_pred = regr.predict(X_test)
        residuals = r2_score(y_test, y_pred)

        # Compares and tracks the best variances
        for x in range(0,len(best)):

            if residuals > best[x]:
                best[x] = residuals
                bestLs[x] = ps
                coef[x] = regr.coef_
                break

In [11]:
# Now lets look at the resulting variance scores
for x in range(0,len(best)):
    print(best[x])

0.940919486567
0.940740800093
0.935192176281
0.934704981238
0.933375784839
0.931949972536
0.931707708153
0.931707060208
0.931353160064
0.930447728321
0.930270861891
0.92198441562
0.918260985383
0.912043773758
0.911094817167
0.910982731887
0.910545559937
0.910436807029
0.909595880289
0.909318036852


In [12]:
# We get scores from 94% to 90% in the top 20. Any of these would work for our model, but we will go through and pick
# the one that makes the most sense heuristically with our League knowledge to ensure that the model won't incorrectly
# predict data down the road. 

# This will show us the variance score, what variables it included and what the weights are for them.
for x in range(0,len(coef)):
    print("Var: " + str(best[x]))
    for y in range(0,len(coef[x])):
        print(bestLs[x][y] + ": " + str(coef[x][y]))
        
    print()

Var: 0.940919486567
Kill Participation: 0.418700301903
Kills: -5.16785359036
Gold: 33.7232065123
Gold Share: -16.9243804411
CS: -1.36556754429
CSPM: 39.0278139619

Var: 0.940740800093
GPM: 0.303157161575
Kill Participation: 0.457150749307
Kills: -5.11081697126
Gold: 24.1853257022
Gold Share: -16.8753026582
CS: -0.943283425008
CSPM: 25.4547151316

Var: 0.935192176281
Kills: -6.13753753005
Gold: 36.2673887188
Gold Share: -15.6903854519
CS: -1.4357446915
CSPM: 37.9178569846

Var: 0.934704981238
GPM: -0.10315729985
Kills: -6.12664451496
Gold: 39.4334046626
Gold Share: -15.7456460634
CS: -1.57724517601
CSPM: 42.5711500044

Var: 0.933375784839
KDA: 1.81620225498
Kill Participation: 0.325639972329
Kills: -5.52049460766
Gold: 31.5349334345
Gold Share: -14.6649990835
CS: -1.28780694922
CSPM: 35.6017579139

Var: 0.931949972536
KDA: 2.09811734882
GPM: 0.593191309816
Kill Participation: 0.386431392477
Kills: -5.46362811076
Gold: 12.5323771206
Gold Share: -14.2182616141
CS: -0.449448292041
CSPM: 8.

In [37]:
# TOP MODEL

from sklearn import linear_model
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Look at the top 20 results
numResults = 20

#These are the possible column variables to be tested in combinations
vars = ['KDA', 'GPM', 'Kill Share', 'Kill Participation', 'Kills', 'Deaths', 'Assists','Gold', 'Gold Share', 'CS', 'CSPM']

best = [-1]*numResults   # Array of top 20 variance scores
bestLs = [-1]*numResults # Array of corresponding variables to each of the variance scores
coef = [[-1]]*numResults # Array of weights associated with each of the variables producing the variance scores

y = mid['Winrate']

for ps in gen_powerset(vars):
    if ps:
        X = mid[ps]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        regr = linear_model.LinearRegression()
        regr.fit(X_train, y_train)
        y_pred = regr.predict(X_test)
        residuals = r2_score(y_test, y_pred)

        # Compares and tracks the best variances
        for x in range(0,len(best)):

            if residuals > best[x]:
                best[x] = residuals
                bestLs[x] = ps
                coef[x] = regr.coef_
                break
                
for x in range(0,len(coef)):
    print("Var: " + str(best[x]))
    for y in range(0,len(coef[x])):
        print(bestLs[x][y] + ": " + str(coef[x][y]))
        
    print()

Var: 0.929553954255
GPM: 0.8807109832
Kill Participation: 0.0880375016736
Deaths: -8.46082065711
Gold Share: -12.7299746083
CSPM: -6.04525122636

Var: 0.928197157877
GPM: 1.01725592525
Kill Participation: 0.0861717354432
Kills: -3.14734880655
Deaths: -8.70156613402
Gold Share: -11.7461412265
CSPM: -10.4016146423

Var: 0.927836759332
GPM: 0.870392534138
Kill Participation: 0.0752072372698
Deaths: -8.59345086144
Assists: 0.233670501155
Gold Share: -12.5094473797
CSPM: -5.93416857338

Var: 0.927710436051
GPM: 1.01196439621
Kill Participation: 0.0809373686686
Kills: -3.122681981
Deaths: -8.75393967343
Assists: 0.0955969376067
Gold Share: -11.6636319644
CSPM: -10.3220273412

Var: 0.926924137369
GPM: 0.927287773474
Kill Participation: 0.0248583942471
Deaths: -7.59575321969
Assists: -0.21124903655
Gold Share: -12.7320689371
CS: 0.0687842085821
CSPM: -9.82954905964

Var: 0.92674512999
GPM: 1.08999048964
Kill Participation: 0.0279113816745
Kills: -3.50457489299
Deaths: -7.70893341505
Assists: -

In [ ]:
#  Top 1
# Var: 0.931949972536
# KDA: 2.09811734882
# GPM: 0.593191309816
# Kill Participation: 0.386431392477
# Kills: -5.46362811076
# Gold: 12.5323771206
# Gold Share: -14.2182616141
# CS: -0.449448292041
# CSPM: 8.51130983888

#  Top 2
# Var: 0.930447728321
# GPM: 0.666826489791
# Kill Participation: 0.46422226111
# Kills: -4.13177746471
# Deaths: -4.54386976931
# Gold: 11.1218003246
# Gold Share: -13.4026852617
# CS: -0.396082068987
# CSPM: 3.59565759673

#  Jungle
# Var: 0.934277194926
# KDA: 0.181164206403
# GPM: 0.735342254784
# Kills: -2.68009575084
# Deaths: -7.25541495648
# Assists: 1.85733953438
# Gold: 7.16660335062
# Gold Share: -14.1978168455
# CS: -0.385377553174
# CSPM: 10.493478078

#  Mid
# GPM: 0.0713156133641
# Deaths: -13.0255264467
# Assists: 3.69342852617
# Gold: 19.8616597461
# CS: -0.845745453945
# CSPM: 12.7679155894

#  ADC
# Var: 0.933270580443
# KDA: 0.879083138604
# GPM: 1.13044121708
# Kill Share: 1.03542306739
# Kill Participation: -0.335817323575
# Kills: -12.9697993181
# Assists: 4.65262777296
# CS: 0.128173966315
# CSPM: -11.2995505544
# Gold Share: -12.5435852695

#  Supp
# Var: 0.915741666545
# KDA: 0.0316139479741
# GPM: 1.338934096
# Kill Share: -0.145858503846
# Kills: -3.99163808505
# Deaths: -6.05433184202
# Gold Share: -23.0728233467

In [60]:
# Use the weights and variables to calculate a player's score and add it as a column.
# Compare 2 teams (Fnatic and IG) and compute the difference of each laner's score.
# Save those differences as variables -> new set of x vars to use
# Somehow have the y-set be the result of the match -> like positive or negative 1 idk bruh

In [47]:
# Here we will use the weights we got from our machine learning and use them to calculate a player's winrate
top['CalculatedWinrate'] = 2.09811734882*top['KDA'] + 0.593191309816*top['GPM'] + 0.386431392477*top['Kill Participation'] + -5.46362811076*top['Kills'] + 12.5323771206*top['Gold'] + -14.2182616141*top['Gold Share'] + -0.449448292041*top['CS'] + 8.51130983888*top['CSPM']
jg['CalculatedWinrate'] = 0.181164206403*jg['KDA'] + 0.735342254784*jg['GPM'] + -2.68009575084*jg['Kills'] + -7.25541495648*jg['Deaths'] + 1.85733953438*jg['Assists'] + 7.16660335062*jg['Gold'] + -14.1978168455*jg['Gold Share'] + -0.385377553174*jg['CS'] + 10.493478078*jg['CSPM']
mid['CalculatedWinrate'] = mid['GPM']*0.0713156133641 + mid['Deaths']*-13.0255264467 + mid['Gold']*19.8616597461 + mid['CS']*-0.845745453945 + mid['CSPM']*12.7679155894 + mid['Assists']*3.69342852617
#mid['CalculatedWinrate2'] = 0.8807109832*mid['GPM'] + 0.0880375016736*mid['Kill Participation'] + -8.46082065711*mid['Deaths'] + -12.7299746083*mid['Gold Share'] + -6.04525122636*mid['CSPM']
adc['CalculatedWinrate'] = 1.13044121708*adc['GPM'] + 1.03542306739*adc['Kill Share'] + -0.335817323575*adc['Kill Participation'] + -12.9697993181*adc['Kills'] + 4.65262777296*adc['Assists'] + 0.128173966315*adc['CS'] + -11.2995505544*adc['CSPM'] + -12.5435852695*adc['Gold Share']
supp['CalculatedWinrate'] = 1.338934096*supp['GPM'] + -0.145858503846*supp['Kill Share'] + -3.99163808505*supp['Kills'] + -6.05433184202*supp['Deaths'] + -23.0728233467*supp['Gold Share']


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [128]:
teams = {}
nadd = [[],[],[],[],[]]

for index,row in masterMatch.iterrows():
    
    x1 = []
    x2 = []
    
    # If we haven't seen this team yet, add it to the dictionary
    if not row['Team 0'] in teams:
        tmp = []
        tmp.append(top[top['Team']==row['Team 0']])
        tmp.append(jg[jg['Team']==row['Team 0']])
        tmp.append(mid[mid['Team']==row['Team 0']])
        tmp.append(adc[adc['Team']==row['Team 0']])
        tmp.append(supp[supp['Team']==row['Team 0']])
        
        for x in tmp:
            if len(x) == 1:
                x1.append(x['CalculatedWinrate'])
            else:
                x1.append(x['CalculatedWinrate'].mean())
                
        teams[row['Team 0']] = x1
        
    else:
        x1 = teams[row['Team 0']]
    
    if not row['Team 1'] in teams:
        tmp = []
        tmp.append(top[top['Team']==row['Team 1']])
        tmp.append(jg[jg['Team']==row['Team 1']])
        tmp.append(mid[mid['Team']==row['Team 1']])
        tmp.append(adc[adc['Team']==row['Team 1']])
        tmp.append(supp[supp['Team']==row['Team 1']])
        
        for x in tmp:
            if len(x) == 1:
                x2.append(x['CalculatedWinrate'])
            else:
                x2.append(x['CalculatedWinrate'].mean())
                
        teams[row['Team 1']] = x2
    
    else:
        x2 = teams[row['Team 1']]
       
    for x in range(0,5):
        nadd[x].append(float(x1[x]) - float(x2[x]))
    
masterMatch['TopDiff'] = nadd[0]
masterMatch['JGDiff'] = nadd[1]
masterMatch['MidDiff'] = nadd[2]
masterMatch['ADCDiff'] = nadd[3]
masterMatch['SuppDiff'] = nadd[4]

In [253]:
# Simulate matches! Just enter 2 teams and look at the result! It's that easy!
# Negative means the team on the right will win. Positive means the team on the left will win

l = teams['Cloud9']
l2 = teams['Invictus_Gaming']
t = l[0] - l2[0]
j = l[1] - l2[1]
m = l[2] - l2[2]
a = l[3].values[0] - l2[3].values[0]
s = l[4].values[0] - l2[4].values[0]

s
t.values[0]*-0.12769662 + j.values[0]*-0.09652615 + m.values[0]*0.00205015 + a*0.28178966  + s*1.12903653

-14.002610898501878

In [ ]:
# Elimination Round Results:

# Africa Freecs --
#                 >---Cloud9---
# Cloud 9 --------             >
#                              >-------Cloud9--------
# Fnatic ---------             >                     >
#                 >---Fnatic---                      >
# EDG ------------                                   >
#                                        -14         >---Invictus Gaming !!
# KT Rolster -----                                   >
#     28          >---Inv. G---                      >
# Invictus Gaming-             >                     >
#                      19      >---Invictus Gaming---
# Royal Never G.--             >
#     -4          >----RNG-----
# G2 Esports -----


In [131]:
tester = masterMatch

vars = ['TopDiff','JGDiff','MidDiff','ADCDiff','SuppDiff']

y = tester['Winner']

X = tester[vars]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
residuals = r2_score(y_test, y_pred)
                
print("Var: " + str(residuals))
print(regr.coef_)
#for y in range(0,len(regr.coef_[x])):
#    print(vars[x] + ": " + str(regr.coef_[x]))       
#    print()

Var: 0.300341143566
[-0.12769662 -0.09652615  0.00205015  0.28178966  1.12903653]


In [82]:
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Look at the top 20 results
numResults = 20

# Must have played at least 7 games
tmp = supp[supp['Games'] >= 7]

#These are the possible column variables to be tested in combinations
vars = ['KDA', 'GPM', 'Kill Share', 'Kill Participation', 'Kills', 'Deaths', 'Assists','Gold', 'CS', 'CSPM', 'Gold Share']
#vars = ['KDA', 'GPM', 'Kill Participation', 'Kills', 'Deaths', 'Assists','Gold']

best = [-1]*numResults   # Array of top 20 variance scores
bestLs = [-1]*numResults # Array of corresponding variables to each of the variance scores
coef = [[-1]]*numResults # Array of weights associated with each of the variables producing the variance scores

y = tmp['Winrate']

for ps in gen_powerset(vars):
    if ps:
        X = tmp[ps]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        regr = linear_model.LinearRegression()
        regr.fit(X_train, y_train)
        y_pred = regr.predict(X_test)
        residuals = r2_score(y_test, y_pred)

        # Compares and tracks the best variances
        for x in range(0,len(best)):

            if residuals > best[x]:
                best[x] = residuals
                bestLs[x] = ps
                coef[x] = regr.coef_
                break
                
for x in range(0,len(coef)):
    print("Var: " + str(best[x]))
    for y in range(0,len(coef[x])):
        print(bestLs[x][y] + ": " + str(coef[x][y]))
        
    print()

Var: 0.93303368213
GPM: 0.959892697284
Kill Share: -1.93590136819
Deaths: -9.68054767182

Var: 0.932813273014
GPM: 0.961272625161
Kill Share: -1.82116071312
Kill Participation: -0.570639772649
Deaths: -10.750655014

Var: 0.92540351474
GPM: 0.948492221554
Kill Share: -1.31576445419
Deaths: -12.8522189351
CS: -0.497250062418

Var: 0.9249796635
KDA: 0.308834421642
GPM: 0.932676204654
Kill Share: -1.29986451328
Deaths: -12.4483953784
CS: -0.488297977327

Var: 0.915779174637
GPM: 1.33949223
Kill Share: -0.155024079963
Kills: -3.91871139229
Deaths: -6.0965931161
Gold Share: -23.0669880678

Var: 0.915741666545
KDA: 0.0316139479741
GPM: 1.338934096
Kill Share: -0.145858503846
Kills: -3.99163808505
Deaths: -6.05433184202
Gold Share: -23.0728233467

Var: 0.915219316028
GPM: 1.31049503351
Kill Share: -0.579412330812
Kill Participation: 0.0821661299183
Deaths: -6.2386070107
Assists: -0.270345362272
Gold Share: -22.8789895483

Var: 0.915190660611
KDA: 0.0400580138009
GPM: 1.30972148663
Kill Share: 